In [ ]:
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)


# Import other libraries
import cv2
from tensorflow.keras.applications.mobilenet import preprocess_input
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

IMAGE PREPROCESSING

In [ ]:
# Load images from the 'items/' folder
img = np.load('data.npy')
label = np.load('labels.npy', allow_pickle=True)
  # Normalize image pixel values to a float range [0, 1]
img = (img / 255).astype(np.float32)

In [ ]:
# set 0,1 label
for i in range(len(label)):
    if(label[i] == 'healthy'):
        label[i] = 1
    else:
        label[i] = 0


In [ ]:
# cleaning images from trol and shrek
ref_img = img[58]
ref_img2 = img[2150]
c = 0
c2 = 0
rm_indexes = []
rm2_indexes = []
for i in range(0, len(img)):
    deviation = np.mean(np.abs(ref_img - img[i]))
    deviation2 = np.mean(np.abs(ref_img2 - img[i]))
    if(deviation == 0.0):
        #print(i)
        c += 1
        rm_indexes.append(i)
    elif(deviation2 == 0.0):
        c2 += 1
        rm2_indexes.append(i)

In [ ]:
clean_img = np.delete(img, rm_indexes + rm2_indexes, axis=0)
clean_label = np.delete(label, rm_indexes + rm2_indexes, axis=0)
print(clean_img.shape)

In [ ]:
# Concatenate 'animals' and 'items' arrays along axis 0
X = clean_img
# Create labels: 1 for 'animals', 0 for 'items'
y = clean_label

#y = np.concatenate([, np.zeros(len(items))], axis=0)
y = tfk.utils.to_categorical(y,len(np.unique(y)))

# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=.25, stratify=np.argmax(y, axis = 1))
# METTERE TEST_SIZE = 15

# Further split train_val into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=len(X_test), stratify=np.argmax(y_train_val, axis = 1))

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

In [ ]:
# Define input shape, output shape, batch size, and number of epochs
input_shape = X_train.shape[1:]
output_shape = y_train.shape#[1:]
batch_size = 32
epochs = 20

# Print input shape, batch size, and number of epochs
print(f"Input Shape: {input_shape}, Output Shape: {output_shape}, Batch Size: {batch_size}, Epochs: {epochs}")

RESNETV50

In [ ]:
resnet = tf.keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=(96, 96, 3),
    pooling='avg',
    classes=2,
    classifier_activation="softmax",
)
resnet.trainable = False
efficent.trainable = False

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [ ]:
img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
    ],
    name="img_augmentation",
)

In [ ]:
# RESNET TRANSFER LEARNING
img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
    ],
    name="img_augmentation",
)

efficent.trainable = False
# Build the neural network layer by layer
inputs = tfkl.Input(shape=(96, 96, 3))
x = img_augmentation(inputs)

# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = resnet(inputs)

# A Dense classifier with a single unit (binary classification)
x  = tfkl.Dense(units=128, kernel_initializer=tf.initializers.HeUniform(seed=seed), activation='relu',name='Output1')(x)
x =  layers.Dropout(0.2)(x)

output_layer = tfkl.Dense(units=2, activation='softmax',name='Output2')(x)
outputs = tfkl.Dense(units=2, activation='softmax',name='Output2')(x)
model = tfk.Model(inputs, outputs)

In [ ]:
callbacks = [
    tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=100, restore_best_weights=True, mode='auto'),
    tf.keras.callbacks.ModelCheckpoint(filepath="/home/andrea/Scrivania/ANN/ch1/checkpoint/",
                                                  save_weights_only=True,
                                                  verbose=1)
]
model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics=['accuracy'])
model.summary()


In [ ]:
# Train the model
tl_history = model.fit(
    x = X_train, # We need to apply the preprocessing thought for the MobileNetV2 network
    y = y_train,
    batch_size = 32,
    epochs = 100,
    validation_data = (X_val, y_val), # We need to apply the preprocessing thought for the MobileNetV2 network
    callbacks = callbacks
).history

In [ ]:
model.save('ResenetV50')
del model